## Análise de homologias por BLAST

1. [local_blastp(tag_to_files, db)](#local)
2. [docker_blastp(directory, db)](#docker)
3. [expasy_blastp(tag_to_files)](#expasy)
4. [blastp(tags_and_proteins, db, type)](#blast)

O objectivo desta fase era, dada as proteínas que retirámos do ficheiro genbank, procurar sequências homólogas com BLAST. Esta fase foi iterada várias vezes à procura da melhor estratégia, e no final acabámos com várias alternativas para a execução do BLAST, que podem ser encontradas no módulo [`util.blast`](https://github.com/vitorenesduarte/bionella/blob/master/util/blast.py).

- __Local__
- __Docker__
- __ExPASy__

<a name="local"></a>
### Local

Esta função assume que o BLAST está instalado localmente, e recebe como parâmetro qual a base de dados que deve usar. O primeiro argumento desta função `tag_to_files` é uma lista de pares em que a primeira componente é o ficheiro onde se encontra a sequência de aminoácidos para ser usada na query, e a segunda componente é o ficheiro onde deve ser guardado o resultado do BLAST.

```python
def local_blastp(tag_to_files, db):
    """
    Corre o blast localmente.
    """

    for tag in tag_to_files:
        (in_file, out_file) = tag_to_files[tag]
    
        blastp_cline = NcbiblastpCommandline(
            query=in_file,
            db=db,
            evalue=10,
            outfmt=5,
            out=out_file
        )
        blastp_cline()
```

<a name="docker"></a>
### Docker

Para evitar a instalação de BLAST, download de bases de dados e sua adição como base de dados para ser usada pelo BLAST, criámos uma imagem __Docker__ com a base de dados __UniProtKB - Swiss-Prot__ [retirada do site da UniProt](http://www.uniprot.org/downloads). Mais informação sobre esta imagem pode ser encontrada [aqui](docker.html).

```python
def docker_blastp(directory, db):
    """
    Corre o blast numa instância docker.
    """
    cmd = "docker run -e QUERY_DIR=" + directory \
                  + " -e DB=" + db \
                  + " -v $PWD/" + directory + ":/" + directory \
                  + " -ti vitorenesduarte/swissprot_blast"

    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    p.wait()
```

<a name="expasy"></a>
### ExPASy

A terceira opção não corre o BLAST localmente, mas utiliza o serviço disponibilizado pelo site ExPASy.

```python
def expasy_blastp(tag_to_files):
    """
    Corre o blast na expasy.
    """
    for tag in tag_to_files:
        (in_file, out_file) = tag_to_files[tag]
        query = rw.read_file(in_file)
        blast_result = www.expasy_blast(query)
        rw.write_file(blast_result, out_file)
```

Esta função usa uma outra função definida no módulo [`util.www`](https://github.com/vitorenesduarte/bionella/blob/master/util/www.py) que corre o BLAST contra a base de dados __UniProtKB = Swiss-Prot + TrEMBL__.

```python
def expasy_blast(query):
    """
    Corre o blast na expasy.
    """

    url = "http://web.expasy.org/cgi-bin/blast/blast.pl"
    data = {
        "format": "xml",
        "prot_db1": "UniProtKB",
        "matrix": "BLOSUM62",
        "showsc": 50, # best scoring sequences
        "showal": 50, # best alignments
        "seq": query
    }

    response = requests.post(
        url,
        data=data
    )

    return response.text
```

Esta opção permitiu-nos encontrar sequências com maior homologia do que as que estávamos a encontrar ao usar apenas a __Swiss-Prot__ localmente.

<a name="blast"></a>
### BLAST

A função que engloba estas três opções recebe como argumento uma lista de pares em que a primeira componente é a __locus_tag__ da proteína, e a segunda componente a respectiva sequência de aminoácidos. Recebe ainda qual o tipo de BLAST, __local__, __docker__ ou __expasy__ a utilizar.

```python
def blastp(tags_and_proteins, db, type="local"):
    """
    Corre o blast para a proteínas passadas como argumento,
    contra a base de dados também passada como argumento.

    O argumento type é opcional e pode ter dois valores:
        - local
        - docker
        - expasy

    É retornado um dicionário com os resultados.
    """

    directory = ".query_dir"
    tag_to_files = write_queries_to_dir(tags_and_proteins, directory)

    # correr o blast
    if type == "local":
        local_blastp(tag_to_files, db)
    elif type == "docker":
        docker_blastp(directory, db)
    elif type == "expasy":
        expasy_blastp(tag_to_files)
    else:
        raise Exception("Unsupported type: " + type)

    blast_results = extract_blast_info(tag_to_files, type)
    return blast_results
```